# **Modelling and Evaluating - RGB**

## Objectives
To develop, train, and evaluate a machine learning model for image classification, incorporating image augmentation and comprehensive performance analysis through various metrics and visualizations. The objective includes modifying class indices, plotting augmented images, creating and summarizing the model, and saving the trained model. Model performance will be assessed using accuracy, ROC curves, and classification reports, followed by plotting the confusion matrix and saving the evaluation results. The final output will also involve making predictions on a random image file.

## Inputs
* Images are taken from the test, train, validation folders and their subfolders.
```
└───inputs/ 
    └───potato_disease_dataset/ 
        ├───test/
        │   ├───healthy
        │   ├───early_blight
        │   └───late_blight                   
        ├───train/
        │   ├───healthy
        │   ├───early_blight
        │   └───late_blight          
        └───validation/
            ├───healthy
            ├───early_blight
            └───late_blight               
```
* Image shape embeddings.


## Outputs
- Image augmentation.
    - Plot augmented images for each dataset.
- Modify class indices to alter prediction in labels.
- Create a machine learning model and display its summary.
    - Train the model.
    - Save the model.
    - Plot the learning curve to show model performance.
        - Model A - generate separate plots for accuracy and loss.
        - Model B - create a comprehensive model history plot.
        - Model C - visualize model history using Plotly.
- Evaluate the model using a saved file.
    - Calculate accuracy.
    - Plot the ROC curve.
    - Generate a general classification report.
    - Generate a classification report including macro average and weighted average.
    - Plot the confusion matrix.
    - Save the evaluation results in a pickle file.
- Predict on a random image file.

---

## Import necessary packages for this notebook

In [ ]:
# Operating system functionality
import os
import pathlib

# Data manipulation and analysis
import pandas as pd
import numpy as np
import random

# Plotting and visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Machine learning and deep learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model

# Model persistence
import joblib

# Scikit-learn for evaluation metrics
import sklearn
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, roc_curve, auc, roc_auc_score

# Inline plotting for Jupyter notebooks
%matplotlib inline

---

## Set Working Directory

As the notebooks are within a subfolder, we will need to change the working directory when running the notebook in the editor.

We need to change the working directory from its current folder to its parent folder.
* We access the current directory with `os.getcwd()`

In [ ]:
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory.
* `os.path.dirname()` gets the parent directory
* `os.chir()` defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory.

In [ ]:
current_dir = os.getcwd()
current_dir

---

## Set input directory
Set train, validation and test paths.

In [ ]:
my_data_dir = 'inputs/potato_disease_dataset'
train_path = my_data_dir + '/train' 
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'

print("Train path:", train_path)
print("Validation path:", val_path)
print("Test path:", test_path)

---

## Set output directory

Change the version number below to change output dir.

In [ ]:
version = 'v3'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(current_dir) and version in os.listdir(current_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

---

## Set Label Names

In [ ]:
labels = os.listdir(train_path)
print('Labels for the images are', labels)

---

### Set image shape

In [ ]:
# Import saved image shape embedding
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")
image_shape

---

## Image data augmentation

### Initialize ImageDataGenerator

In [ ]:
augmented_image_data = ImageDataGenerator(rotation_range=20,
                                          width_shift_range=0.10,
                                          height_shift_range=0.10,
                                          shear_range=0.1,
                                          zoom_range=0.1,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          fill_mode='nearest',
                                          rescale=1./255
                                          )

### Set Batch Size

In [ ]:
batch_size = 16

### Augment datasets

#### Augment training image set

In [ ]:
train_set = augmented_image_data.flow_from_directory(train_path,
                                                     target_size=image_shape[:2],
                                                     color_mode='rgb',
                                                     batch_size=batch_size,
                                                     class_mode='categorical',
                                                     shuffle=True
                                                     )

train_set.class_indices

#### Augment validation image set

In [ ]:
validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
                                                                        target_size=image_shape[:2],
                                                                        color_mode='rgb',
                                                                        batch_size=batch_size,
                                                                        class_mode='categorical',
                                                                        shuffle=False
                                                                        )

validation_set.class_indices

#### Augment test image set

In [ ]:
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                                  target_size=image_shape[:2],
                                                                  color_mode='rgb',
                                                                  batch_size=batch_size,
                                                                  class_mode='categorical',
                                                                  shuffle=False
                                                                  )

test_set.class_indices

### Plot augmented datasets

In [ ]:
# Function to plot augmented datasets
def plot_augmented_images(dataset, title):
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(6, 6))
    for i in range(3):
        for j in range(3):
            img, label = dataset.next()
            print(img.shape)
            axes[i, j].imshow(img[0])
            axes[i, j].axis('off')
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

#### Plot augmented training image set

In [ ]:
# Plot augmented training image set
plot_augmented_images(train_set, "Augmented Training Images")

#### Plot augmented validation image set

In [ ]:
# Plot augmented validation image set
plot_augmented_images(validation_set, "Augmented Validation Images")

### Plot augmented test image set

In [ ]:
# Plot augmented test image set
plot_augmented_images(test_set, "Augmented Test Images")

### Save class indices

In [ ]:
joblib.dump(value=train_set.class_indices,
            filename=f"{file_path}/class_indices.pkl")

---

## Model Creation

### Define the CNN Model

In [ ]:
model = Sequential()

# First Convolutional Layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=image_shape))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

# Second Convolutional Layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

# Third Convolutional Layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

# Flatten the layers
model.add(Flatten())

# Fully Connected Layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

# Output Layer
model.add(Dense(3, activation='softmax'))  # Assuming 3 classes for potato leaf diseases

# Summary of the custom model
model.summary()

### Compile the model

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

### Callbbacks

In [ ]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define model checkpoint callback
checkpoint = ModelCheckpoint(f"{file_path}/best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max')

# Define the learning rate reduction callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)

### Train the model

In [ ]:
model.fit(
        train_set,
        epochs=50,
        validation_data=validation_set,
        callbacks=[early_stopping, checkpoint, reduce_lr])

### Load saved best model

In [ ]:
load_model(f"{file_path}/best_model.h5")

---

## Model Performance

### Model Learning Curve - A
This cell creates and saves plots of the training and validation loss and accuracy.

In [ ]:
# Plotting training and validation loss
losses = pd.DataFrame(model.history.history)
plt.figure(figsize=(5, 4))
sns.set_style("whitegrid")
losses[['loss', 'val_loss']].plot(style='.-')
plt.title("Loss")

# Save the plot of training and validation loss
plt.savefig(f'{file_path}/curve_a_model_training_losses.png', bbox_inches='tight', dpi=150)

print("\n")

# Plotting training and validation accuracy
losses[['accuracy', 'val_accuracy']].plot(style='.-')
plt.title("Accuracy")

# Save the plot of training and validation accuracy
plt.savefig(f'{file_path}/curve_a_model_training_acc.png', bbox_inches='tight', dpi=150)

plt.show()

### Model Learning Curve - B
This cell generates and saves a single combined plot of all training and validation metrics.

In [ ]:
# Plotting training and validation metrics together
pd.DataFrame(model.history.history).plot(figsize=(8, 5))

# Save the combined plot of all training metrics
plt.savefig(f'{file_path}/curve_b_model_loss_acc.png', bbox_inches='tight', dpi=150)
plt.show()

### Model Learning Curve - C

This cell creates an interactive plot with Plotly to show training and validation loss and accuracy on a dual y-axis, and saves the plot as an image.

In [ ]:
# Create subplots with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add validation loss trace to the plot
fig.add_trace(
    go.Scatter(y=model.history.history['val_loss'], name="val_loss"),
    secondary_y=False,
)

# Add training loss trace to the plot
fig.add_trace(
    go.Scatter(y=model.history.history['loss'], name="loss"),
    secondary_y=False,
)

# Add validation accuracy trace to the plot
fig.add_trace(
    go.Scatter(y=model.history.history['val_accuracy'], name="val accuracy"),
    secondary_y=True,
)

# Add training accuracy trace to the plot
fig.add_trace(
    go.Scatter(y=model.history.history['accuracy'], name="accuracy"),
    secondary_y=True,
)

# Update layout with title and axis labels
fig.update_layout(
    title_text="Loss/Accuracy of Model"
)

fig.update_xaxes(title_text="Epoch")

fig.update_yaxes(title_text="<b>primary</b> Loss", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> Accuracy", secondary_y=True)

fig.update_layout(
    autosize=False,
    width=800, 
    height=500, 
    )

# Show the plot
fig.show()

---

## Model Evaluation

### Load saved model
Loads the trained and saved model from the specified file path.

In [ ]:
best_model = load_model(f"{file_path}/best_model.h5")

### Evaluate the model on the test set

In [ ]:
test_loss, test_accuracy = best_model.evaluate(test_set)
print(f"Test Accuracy: {test_accuracy}")

### Calculate accuracy and generate predictions

Reset the test set iterator, calculates the true labels, and generates predictions for the test set.

In [ ]:
# Reset the test set iterator
test_set.reset()

# Generate predictions
y_true = test_set.classes
y_pred = best_model.predict(test_set, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)

### Generate classification report - A

Generates and prints a standard classification report.

In [ ]:
general_report = classification_report(y_true, y_pred_classes, target_names=labels)
print("Classification Report for Model A:\n", general_report)

### Generate classification report - B

Generates a classification report and visualizes it as a heatmap

In [ ]:
classification_report_heatmap = classification_report(y_true, y_pred_classes, target_names=labels, output_dict=True)
fig, ax = plt.subplots(figsize=(8, 5))
sns.heatmap(pd.DataFrame(classification_report_heatmap).iloc[:-1, :].T, annot=True, cmap="Blues", cbar=False, linewidths=1)
plt.title('Classification Report')
plt.savefig(f'{file_path}/classification_report_heatmap.png')
plt.show()

### Plot ROC curve

Plots and saves the ROC curve for each class in the test set.

In [ ]:
true_positive_rate = {}
false_positive_rate = {}
roc_auc = {}
for i, label in enumerate(labels):
    false_positive_rate[label], true_positive_rate[label], _ = roc_curve(y_true == i, y_pred[:, i])
    roc_auc[label] = auc(false_positive_rate[label], true_positive_rate[label])

plt.figure()
for label in labels:
    plt.plot(false_positive_rate[label], true_positive_rate[label], label=f'ROC curve (area = {roc_auc[label]:.2f}) for {label}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.savefig(f'{file_path}/roc_curve.png')
plt.show()

### Plot Confusion Matrix

plots and saves the confusion matrix.

In [ ]:
# Plot Confusion Matrix
confusion_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig(f'{file_path}/confusion_matrix.png')
plt.show()

### Save the evaluation results in a pickle file

Saves the evaluation results, including accuracy, classification reports, and the confusion matrix, into a pickle file.

In [ ]:
evaluation_results = {
    "test_accuracy": test_accuracy,
    "general_report": general_report,
    "classification_report_heatmap": classification_report_heatmap,
    "confusion_matrix": confusion_matrix,
    "roc_curve": {
    "true_positive_rate": true_positive_rate,
    "false_positive_rate": false_positive_rate,
    "roc_auc": roc_auc
}
}
joblib.dump(evaluation_results, f"{file_path}/evaluation_results.pkl")

---

## Predict on new data

In [ ]:
# Select a random image from the test set
label = random.choice(labels)
image_path = os.path.join(test_path, label)
image_file = random.choice(os.listdir(image_path))

In [ ]:
# Load the image
pil_image = load_img(os.path.join(image_path, image_file), target_size=image_shape[:2], color_mode='rgb')
print(f'Image shape: {pil_image.size}, Image mode: {pil_image.mode}')
plt.imshow(pil_image)
plt.title(f'Random Image from {label}')
plt.show()

# Convert the image to an array and preprocess it
my_image = img_to_array(pil_image)
my_image = np.expand_dims(my_image, axis=0) / 255.0
print(my_image.shape)

# Predict class probabilities
pred_proba = best_model.predict(my_image)

# Map predicted class index to class label
target_map = {v: k for k, v in train_set.class_indices.items()}
pred_class_index = np.argmax(pred_proba)
pred_class = target_map[pred_class_index]
pred_proba = pred_proba[0][pred_class_index]

print(f'Predicted Probability: {pred_proba}')
print(f'Predicted Class: {pred_class}')